In [1]:
from torchvision.transforms.v2 import ToTensor

VERSION = '001'
BATCHSIZE = 32
MAX_EPOCHS = 50
BACKBONE = "resnet50"
MODEL_NAME = "ResNet50_transfer"
NUM_WORKERS = 31

In [2]:

import os
from torchvision.transforms import Compose, Resize, InterpolationMode, ToTensor, Normalize, RandomCrop, RandomHorizontalFlip
import torch
from torchvision import transforms
from code.classifier import PneumoniaClassifier, Config
import matplotlib.pyplot as plt
import numpy as np



In [3]:

# Load the TensorBoard extension
%reload_ext tensorboard
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
# Enable CUDA Launch Blocking
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# Set float32 matrix multiplication precision to 'medium' to utilize Tensor Cores
torch.set_float32_matmul_precision('medium')
torch.serialization.add_safe_globals([Compose, Resize, set, Config, InterpolationMode, ToTensor, Normalize, RandomHorizontalFlip])

In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Tue Nov 26 22:57:22 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.90                 Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 Ti   WDDM  |   00000000:02:00.0  On |                  N/A |
|  0%   45C    P8             13W /  285W |    1518MiB /  12282MiB |      6%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
from torchvision import transforms


config = Config(
    backbone_name="resnet18",
    transfer_learning=True,
    learning_rate=1e-4,
    batch_size=20,
    max_epochs=100,
    weight_decay=1e-4,
    dropout=0.5,
    num_workers=31,
    model_name="ResNet18_transfer",
    version="001",
    optimizer_name = "sgd"
)

model = PneumoniaClassifier(config)
model.train_model()

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\aszab\Documents\Studium\Modules\ComputerScienceProject\MedicalImaging\pneumonia_detection\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

  | Name              | Type            | Params | Mode 
--------------------------------------------------------------
0 | accuracy          | BinaryAccuracy  | 0      | train
1 | precision         | BinaryPrecision | 0      | train
2 | recall            | BinaryRecall    | 0      | train
3 | f


Detected KeyboardInterrupt, attempting graceful shutdown ...


RuntimeError: Please call `iter(combined_loader)` first.

In [7]:
checkpoint = torch.load(f"../checkpoints/{config.model_name}.ckpt", weights_only=True)

# Access the metadata
metadata = checkpoint.get("metadata", {})

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL torchvision.transforms.transforms.RandomHorizontalFlip was not an allowed global by default. Please use `torch.serialization.add_safe_globals([RandomHorizontalFlip])` to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with weights_only https://pytorch.org/docs/stable/generated/torch.load.html.

In [8]:
# Initialize the model
model = PneumoniaClassifier(config, transform)
# Test the model
metadata = model.test_model(f"../checkpoints/{config.model_name}.ckpt")

# Print the metadata
print(metadata)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 32/32 [00:05<00:00,  6.13it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_acc_epoch         0.6073718070983887
      test_acc_step         0.48149147629737854
      test_f1_epoch          0.71074378490448
      test_f1_step          0.34881046414375305
        test_loss           0.6804386973381042
  test_precision_epoch      0.6586433053016663
   test_precision_step      0.2879661023616791
    test_recall_epoch        0.771794855594635
    test_recall_step        0.4989185929298401
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
None


In [9]:
def visualize_gradcam(self):
    random_img = self.random_image
    image = self.transform(random_img).unsqueeze(0).to(self.device)

    target_layer = self.feature_extractor[-1]  # Use the last convolutional layer
    cam = self.generate_gradcam(image, target_layer)
    original_img = np.array(random_img) / 255.0  # Normalize original image

    cam_img = self.show_gradcam_on_image(original_img, cam)
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(original_img)
    plt.axis("off")
    plt.subplot(1, 2, 2)
    plt.title("Grad-CAM")
    plt.imshow(cam_img)
    plt.axis("off")
    plt.show()